In [1]:
from pymongo import MongoClient
from pigeonXT import annotate
from pathlib import Path
import pandas as pd
import numpy as np
import random

In [2]:
def hash():
    return "%08x" % random.getrandbits(32)


hash = hash()

In [3]:
MONGO_HOST = "mongodb://localhost/twitterdb"
client = MongoClient(MONGO_HOST)
raw_tweets = client.twitterdb.tweets_raw

In [4]:
raw_df = pd.DataFrame(list(raw_tweets.find()))
raw_df.sample(2, random_state=42)

,_id,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,...,favorite_count,favorited,retweeted,lang,query,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status
1491,62f3fbe2ef45b02417c6518a,Wed Aug 10 04:47:56 +0000 2022,1557227224293597186,1557227224293597186,"@lybrjack Uma boa questão, um amigo meu pergun...",False,"[10, 270]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'pt', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,0,False,False,pt,jeito de gay,NaN,NaN,NaN,NaN,NaN
450,62f3ef217a58ae552109c8fe,Wed Aug 10 14:04:29 +0000 2022,1557367283856596992,1557367283856596992,"@JoaquinTeixeira @jairbolsonaro sei não, esse ...",False,"[32, 73]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'pt', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,0,False,False,pt,cara de macaco,NaN,NaN,NaN,NaN,NaN


In [5]:
public_df = raw_df.copy()[["id", "full_text"]]
public_df.drop_duplicates(inplace=True)
public_df.head()

,id,full_text
0,1555630597527199746,@0ravl4 @hoje_no Além de petralha a safada ain...
1,1557015903224700929,"NÃO existe ""coxinha sem massa"",bando de burro pqp"
2,1556198185059684362,@heldermaldonado Coxinha burro!!
3,1555581110880436224,"Ser petista é ser burro para mas de metro , pa..."
4,1554478359375126528,Burro burro burro burro burro me ofereceram um...


In [6]:
sample = public_df.sample(20)
sample.to_csv(f"../data/input_test_data_{hash}.csv", index=False)
sample.head()

,id,full_text
259,1557422001073037312,eu sou muito boiola por essas duas sim 🥺🤏🏼\n\n...
300,1557213357341646853,"@LeonelRadde E a Rosca, \nQueima ou não Queima..."
977,1557179504908460032,@jandira_feghali Chuta que é macumba
137,1557415987045171202,vai se foder puta desgraçada vagabunda egoísta...
2099,1557434423527849984,me sentindo gord*o


In [7]:
labels = [
    "political",
    "sexism",
    "lgbtphobia",
    "racism",
    "aporophobia",
    "xenophobia",
    "ableism",
    "ageism",
    "religion",
    "body_shame",
    "other",
    "not_applicable",
]

shortLabels = ["PL", "SX", "LGBT", "RC", "AP", "XP", "AB", "AG", "RL", "BS", "OT", "NA"]


def setLabels(labels, numClasses):
    row = np.zeros([numClasses], dtype=np.uint8)
    row[labels] = 1
    return row


def labelPortion(
    inputFile,
    labels=["yes", "no"],
    outputFile=f"../data/output_test_data_{hash}.csv",
    portionSize=20,
    textColumn="full_text",
    shortLabels=None,
):

    if shortLabels:
        shortLabels = shortLabels
    else:
        shortLabels = labels

    out = Path(outputFile)
    if out.exists():
        outdf = pd.read_csv(out)
        currentId = outdf.index.max() + 1
    else:
        currentId = 0

    indf = pd.read_csv(inputFile)
    examplesInFile = len(indf)
    indf = indf.loc[currentId : currentId + portionSize - 1]
    actualPortionSize = len(indf)
    print(f"{currentId + 1} - {currentId + actualPortionSize} of {examplesInFile}")
    sentences = indf[textColumn].tolist()

    for label in shortLabels:
        indf[label] = None

    def updateRow(example, selectedLabels):
        print(example, selectedLabels)
        labs = setLabels([labels.index(y) for y in selectedLabels], len(labels))
        indf.loc[indf[textColumn] == example, shortLabels] = labs

    def finalProcessing(annotations):
        if out.exists():
            prevdata = pd.read_csv(out)
            outdata = pd.concat([prevdata, indf]).reset_index(drop=True)
        else:
            outdata = indf.copy()
        outdata.to_csv(out, index=False)

    _ = annotate(
        sentences,
        options=labels,
        task_type="multilabel-classification",
        buttons_in_a_row=3,
        reset_buttons_after_click=True,
        include_next=False,
        include_back=False,
        example_process_fn=updateRow,
        final_process_fn=finalProcessing,
    )
    return indf


def getAnnotationsCountPerlabel(annotations, shortLabels):

    countPerLabel = pd.DataFrame(columns=shortLabels, index=["count"])

    for label in shortLabels:
        countPerLabel.loc["count", label] = len(annotations.loc[annotations[label] == 1.0])

    return countPerLabel


annotations = labelPortion(f"../data/input_test_data_{hash}.csv", labels=labels, shortLabels=shortLabels)

1 - 20 of 20


HTML(value='0 of 20 Examples annotated, Current Position: 0 ')

Output()

eu sou muito boiola por essas duas sim 🥺🤏🏼

#motherlandforsalem

https://t.co/wax5SnkXof ['not_applicable']


@LeonelRadde E a Rosca, 
Queima ou não Queima??
😎🤣🤣🤣🤣 ['lgbtphobia']


...

hj uma safada queria me passar nota falsa vontade de dar na cara dela q ódio ['sexism']


@reveloud ja pode contar pros seus seguidores q vc eh um velho tarado de 52 anos ['ageism']


@douglasdelete @CLMerlo Pesquisa ai a divida do teu time, vê lá quem deve mais. mongol burro! ['ableism']


vim fazer comprinhas no supermercado e comprei tanta besteira chocolates, doces etc parece que terminei mais um namoro mas é só a solidão do twink favelado mesmo https://t.co/3F2LWE1ezD ['not_applicable']


Já fiz assim, mas eu era jovem e burro. https://t.co/1M3LcbyrVT ['not_applicable']
Annotation done.
